In [1]:

import csv
import pandas as pd
from imblearn.over_sampling import BorderlineSMOTE
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import classification_report, accuracy_score


CSV파일 불러오기

In [2]:
predata = pd.read_csv("./0825finalpredata.csv",encoding='cp949')
predata = pd.DataFrame(predata)

In [4]:
predata.head()

,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,15020,15021,15022,15023,15024,15025,15026,0_y.1,0,Priority
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.395026,1.495279,3
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.661092,0.996102,3
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.205874,0.360785,2
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.661092,0.973412,3
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.205874,-0.433361,3


Input과 label 나누기

In [3]:

# input data 분할
X = predata.iloc[:,0:22321].values
X1 = predata.iloc[:,22321:22323].values
#  input data 전체

X2 = predata.iloc[:,0:22323].values

Y = predata['Priority']
X2


array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.20148581,  0.47748626],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.71456489, -0.24112721],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         1.63061558,  3.3983023 ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.20148581, -0.47293801],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.20148581, -0.47293801],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.20148581, -0.0324975 ]])

Feature selection 500개

In [4]:

chi2_features = SelectKBest(chi2, k = 1000) 
X_kbest_features = chi2_features.fit_transform(X, Y) 

# Reduced features 
print('Original feature number:', X.shape[1]) 
print('Reduced feature number:', X_kbest_features.shape[1])

X_kbest_features = pd.DataFrame(X_kbest_features)
X1 = pd.DataFrame(X1)
X3 = pd.merge(X_kbest_features,X1, how= 'outer', left_index=True, right_index =True)


Original feature number: 22321
Reduced feature number: 1000


Train, Test Data 나누기

In [5]:

#  input data 전체
X_train, X_test, y_train, y_test = train_test_split(X3, Y, test_size=0.3, random_state=0)


Oversampling

In [6]:

smote = BorderlineSMOTE(random_state=42)
new_X_train,new_y_train = smote.fit_resample(X_train,y_train)
print(new_y_train.value_counts())


2.0    2592
5.0    2592
3.0    2592
4.0    2592
1.0    2592
Name: Priority, dtype: int64


RandomForest 적용

In [7]:
#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100, random_state = 42)

#Train the model using the training sets y_pred=clf.predict(X_test)
#clf.fit(new_X_train,new_y_train)
clf.fit(new_X_train,new_y_train)

y_pred=clf.predict(X_test)

# RandomForest 정확도
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


Accuracy: 0.7433547204399633


SVM 적용

In [8]:

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(new_X_train,new_y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)


print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


Accuracy: 0.5916590284142988


XGBoost 적용

In [12]:

import xgboost as xgb
xgb_clf = xgb.XGBClassifier()

# XGBoost 예측모형
xgb_model = xgb.XGBClassifier(silent=False, 
                              verbosity=0,
                              booster='gbtree',
                              scale_pos_weight=1,
                              learning_rate=0.01,  
                              colsample_bytree = 0.4,
                              subsample = 0.8,
                              objective='binary:logistic', 
                              n_estimators=100, 
                              max_depth=5, 
                              gamma=10, 
                              seed=777)

xgb_model.fit(new_X_train, new_y_train)

y_pred = xgb_model.predict(X_test)

print(classification_report(y_test, y_pred))


C:\Users\jhmak\Anaconda3\lib\site-packages\distributed\config.py:63: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config.update(yaml.load(text) or {})


              precision    recall  f1-score   support

         1.0       0.06      0.05      0.05        61
         2.0       0.41      0.36      0.38       310
         3.0       0.88      0.54      0.67      1135
         4.0       0.48      0.85      0.61       654
         5.0       0.00      0.00      0.00        22

    accuracy                           0.59      2182
   macro avg       0.37      0.36      0.35      2182
weighted avg       0.66      0.59      0.59      2182



depth[ 3, 4, 7, 9, 10 ]에 따른 정확도 예측

In [13]:

import time
import progressbar

accuracy_list = []


# max_depth_list = [3,5,7,9,10]
max_depth_list = [3,4,7,9,10]

bar = progressbar.ProgressBar(maxval=10).start()
for max_depth in max_depth_list:
    xgb_model = xgb.XGBClassifier(max_depth=max_depth, seed=777)
    xgb_model.fit(new_X_train,new_y_train)
    y_pred = xgb_model.predict(X_test)
    xgb_accuracy = accuracy_score(y_test, y_pred) 
    accuracy_list.append(xgb_accuracy)
    bar.update(max_depth)
    
bar.finish()
xgb_df = pd.DataFrame({'tree depth':max_depth_list, 'accuracy':accuracy_list})
xgb_df.head()    


100% |########################################################################|


,tree depth,accuracy
0,3,0.615032
1,4,0.634280
2,7,0.674152
3,9,0.702566
4,10,0.706691


KNN 적용 ( k = 5 )

In [9]:

#Create KNN Classifier
knn = KNeighborsClassifier(n_neighbors=5)

#Train the model using the training sets
knn.fit(new_X_train,new_y_train)

#Predict the response for test dataset
y_pred = knn.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


Accuracy: 0.5623281393217232


KNN 적용 ( k = 7 )

In [10]:

#Create KNN Classifier
knn2 = KNeighborsClassifier(n_neighbors=7)

#Train the model using the training sets
knn2.fit(new_X_train,new_y_train)

#Predict the response for test dataset
y_pred = knn2.predict(X_test)


# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


Accuracy: 0.5426214482126489


Decision Tree 적용

In [11]:

# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(new_X_train,new_y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


Accuracy: 0.6837763519706691
